In [2]:
#************************Performance Evaluation******************
import numpy as np

# Build the inception v3 network
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3

model_inception = InceptionV3(include_top=False, weights='imagenet',input_shape=(300, 300, 3))

#model_inception.summary()
    
model = Sequential()
model.add(model_inception)
model.add(Flatten())
model.add(Dense(6, activation='softmax'))

model.load_weights('/Users/apple/Downloads/inception_flatten_dense.h5')
print('Model loaded.')

#compile
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['categorical_accuracy'])
 

Model loaded.


In [36]:
# This part is a little different from the paper.
# I also evaluated the performance of Google Cloud Vision when I was doing the project which is not mentioned in the paper

import io
import os
import sys
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

# Build dictionary for reference
meat_list = ["beef", "chicken", "lamb", "pork", "tuna", "turkey"]

target_size = (300, 300)

#Predict by CNN model 
def cnn_method(path):
    img = load_img(path, target_size = target_size)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    x *= 1. /255
    return meat_list[model.predict_classes(x)[0]]

#Predict on Google Cloud Vision first, and then on our CNN model
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Client's Json file"
# Instantiates a client
client = vision.ImageAnnotatorClient()

def combine_method(path):
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = types.Image(content=content)

    # Performs label detection on the image file
    response = client.label_detection(image=image)
    labels = response.label_annotations

    #Search the label containing the keyword in order
    #If all labels do not contain the keyword, pass the image into the CNN model
    for label in labels:
        label_name = label.description.lower()
        #print(labelname)
        for meat in meat_list:
            if meat in label_name:
                return meat
    return cnn_method(path)
    

In [6]:
#Test on single image
combine_method('/Users/apple/Mcgill/689PROJECT/code/pictures/test/1_beef/000003.jpg')


'beef'

In [47]:
#Iterate pictures in a file and calcuate the recall for each category of meat
def eval_one_category(rel_path, meat_name, method):
    total_pic = 0
    count = 0
    ab_path = os.path.abspath('')
    for filename in os.listdir(ab_path + rel_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"): 
            total_pic += 1 
            file_ab_name = os.path.join(ab_path, rel_path[1:], filename)
            if method(file_ab_name) == meat_name:
                #print(count)
                count += 1
    print("{} accuracy = {:.3f}".format(meat_name, count/total_pic)) 
            

In [61]:
#Evaluate the accuracy of combined method or CNN method performance on one meat category

# For example
eval_one_category("/pictures/test/2_chicken","chicken", cnn_method)

eval_one_category("/pictures/test/2_chicken","chicken", combine_method)


/Users/apple/myenv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag))
/Users/apple/myenv/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


chicken accuracy = 0.477


In [43]:
#Overall model performance: recall and precision
#meat_list = ["beef", "chicken", "lamb", "pork", "tuna", "turkey"]

def eval_overall(method):

    ab_path = os.path.abspath('')
    rel_list = ["/pictures/test/1_beef","/pictures/test/2_chicken",
                "/pictures/test/3_lamb","/pictures/test/4_pork",
                "/pictures/test/5_tuna","/pictures/test/6_turkey"]
    
    #tp_dict is for true positives, and predict_dict is for predicted as positives, actual_dict is for actual labels
    tp_dict = {"beef":0, "chicken":0,"lamb":0, "pork":0,"tuna": 0,"turkey": 0}
    predict_dict = {"beef":0, "chicken":0,"lamb":0, "pork":0,"tuna": 0,"turkey": 0}
    actual_dict = {"beef":0, "chicken":0,"lamb":0, "pork":0,"tuna": 0,"turkey": 0}
    
    for m, p in zip(meat_list, rel_list):
        for filename in os.listdir(ab_path + p):
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"): 
                file_ab_name = os.path.join(ab_path, p[1:], filename)
                actual_dict[m] += 1 # the actual label 
                temp = method(file_ab_name) # the predicted label
                predict_dict[temp] += 1
                if temp == m:
                    tp_dict[m] += 1 # the true positives
    #Print (tp_dict, predict_dict, actual_dict)
    for m in meat_list:
        print("{} precision = {:.3f}".format(m, tp_dict[m]/predict_dict[m])) 
        print("{} recall = {:.3f}".format(m, tp_dict[m]/actual_dict[m]))
      

In [44]:
eval_overall(cnn_method)

/Users/apple/myenv/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/Users/apple/myenv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag))


{'beef': 82, 'chicken': 86, 'lamb': 59, 'pork': 65, 'tuna': 63, 'turkey': 90} {'beef': 148, 'chicken': 150, 'lamb': 85, 'pork': 101, 'tuna': 75, 'turkey': 107} {'beef': 113, 'chicken': 109, 'lamb': 102, 'pork': 134, 'tuna': 101, 'turkey': 107}
beef precision = 0.554
beef recall = 0.726
chicken precision = 0.573
chicken recall = 0.789
lamb precision = 0.694
lamb recall = 0.578
pork precision = 0.644
pork recall = 0.485
tuna precision = 0.840
tuna recall = 0.624
turkey precision = 0.841
turkey recall = 0.841
